# Using TensorFlow for Classification
Let's take a look at a scenario where we can recognize different items of clothing (Fashion MNIST), trained from a dataset containing 10 different types.

## Start Coding

Let's start with our import of TensorFlow

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


The Fashion MNIST data is available directly in the tf.keras datasets API. You load it like this:

In [2]:
mnist = tf.keras.datasets.fashion_mnist

Calling load_data on this object will give you two sets of two lists, these will be the training and testing values for the graphics that contain the clothing items and their labels.


In [3]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [72]:
training_images.shape, training_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

What does these values look like? Let's print a training image, and a training label to see...Experiment with different indices in the array. For example, also take a look at index 42...that's a a different boot than the one at index 0


In [73]:
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

9
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0   0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62  54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134 144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178 107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216 216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   0 200 232 232 233 229 223 

You'll notice that all of the values in the number are between 0 and 255. If we are training a neural network, for various reasons it's easier if we treat all values as between 0 and 1, a process called '**normalizing**'...and fortunately in Python it's easy to normalize a list like this without looping. You do it like this:

In [74]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

In [75]:
print(training_images[0])

[[0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.         0.    

Now you might be wondering why there are 2 sets...training and testing -- remember we spoke about this in the intro? The idea is to have 1 set of data for training, and then another set of data...that the model hasn't yet seen...to see how good it would be at classifying values. After all, when you're done, you're going to want to try it out with data that it hadn't previously seen!

Let's now design the model. There's quite a few new concepts here, but don't worry, you'll get the hang of them. 

In [4]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

**Sequential**: That defines a SEQUENCE of layers in the neural network

**Flatten**: Remember earlier where our images were a square, when you printed them out? Flatten just takes that square and turns it into a 1 dimensional set.

**Dense**: Adds a layer of neurons

Each layer of neurons need an **activation function** to tell them what to do. There's lots of options, but just use these for now. 

**Relu** effectively means "If X>0 return X, else return 0" -- so what it does it it only passes values 0 or greater to the next layer in the network.

**Softmax** takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0] -- The goal is to save a lot of coding!


The next thing to do, now the model is defined, is to actually build it. You do this by compiling it with an optimizer and loss function as before -- and then you train it by calling **model.fit ** asking it to fit your training data to your training labels -- i.e. have it figure out the relationship between the training data and its actual labels, so in future if you have data that looks like the training data, then it can make a prediction for what that data would look like. 

In [7]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(training_images, training_labels, validation_split = 0.1, epochs=5)

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5544 - accuracy: 0.7982 - val_loss: 0.5409 - val_accuracy: 0.8012
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5400 - accuracy: 0.8028 - val_loss: 0.5327 - val_accuracy: 0.7985
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5428 - accuracy: 0.8038 - val_loss: 0.5443 - val_accuracy: 0.8065
Epoch 4/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5321 - accuracy: 0.8051 - val_loss: 0.5458 - val_accuracy: 0.8087
Epoch 5/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5294 - accuracy: 0.8069 - val_loss: 0.5284 - val_accuracy: 0.8130


In [86]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (32, 784)                 0         
_________________________________________________________________
dense_7 (Dense)              (32, 128)                 100480    
_________________________________________________________________
dense_8 (Dense)              (32, 10)                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


Once it's done training -- you should see an accuracy value at the end of the final epoch. It might look something like 0.9098. This tells you that your neural network is about 91% accurate in classifying the training data. I.E., it figured out a pattern match between the image and the labels that worked 91% of the time. Not great, but not bad considering it was only trained for 5 epochs and done quite quickly.

But how would it work with unseen data? That's why we have the test images. We can call model.evaluate, and pass in the two sets, and it will report back the loss for each. Let's give it a try:

In [87]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 558us/step - loss: 0.3541 - accuracy: 0.8719


[0.35406243801116943, 0.8719000220298767]

For me, that returned a accuracy of about .8838, which means it was about 88% accurate. As expected it probably would not do as well with *unseen* data as it did with data it was trained on!  As you go through this course, you'll look at ways to improve this. 

To explore further, try the below exercises:


# Exploration Exercises

###Exercise 1:
For this first exercise run the below code: It creates a set of classifications for each of the test images, and then prints the first entry in the classifications. The output, after you run it is a list of numbers. Why do you think this is, and what do those numbers represent? 

In [100]:
classifications = model.predict(test_images)

print(classifications[0])

[1.38580945e-05 3.99917361e-08 5.54535472e-06 4.71186610e-07 1.31605236e-06 1.28970332e-02 6.82252994e-06 7.82848708e-03 6.36003688e-06 9.79240179e-01]


Hint: try running print(test_labels[0]) -- and you'll get a 9. Does that help you understand why this list looks the way it does? 

In [101]:
print(test_labels[0])

9


### What does this list represent?


1.   It's 10 random meaningless values
2.   It's the first 10 classifications that the computer made
3.   It's the probability that this item is each of the 10 classes


#### Answer: 
The correct answer is (3)

The output of the model is a list of 10 numbers. These numbers are a probability that the value being classified is the corresponding value (https://github.com/zalandoresearch/fashion-mnist#labels), i.e. the first value in the list is the probability that the image is of a '0' (T-shirt/top), the next is a '1' (Trouser) etc. Notice that they are all VERY LOW probabilities.

For the 9 (Ankle boot), the probability was in the 90's, i.e. the neural network is telling us that it's almost certainly a 7.

### How do you know that this list tells you that the item is an ankle boot?


1.   There's not enough information to answer that question
2.   The 10th element on the list is the biggest, and the ankle boot is labelled 9
2.   The ankle boot is label 9, and there are 0->9 elements in the list


#### Answer
The correct answer is (2). Both the list and the labels are 0 based, so the ankle boot having label 9 means that it is the 10th of the 10 classes. The list having the 10th element being the highest value means that the Neural Network has predicted that the item it is classifying is most likely an ankle boot

## Exercise 2: 
Let's now look at the layers in your model. Experiment with different values for the dense layer with 512 neurons. What different results do you get for loss, training time etc? Why do you think that's the case? 


In [39]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1857 - accuracy: 0.9440
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0724 - accuracy: 0.9775
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0476 - accuracy: 0.9848
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0332 - accuracy: 0.9893
Epoch 5/5
313/313 [==============================] - 0s 860us/step - loss: 0.0792 - accuracy: 0.9783


[0.07917152345180511, 0.9782999753952026]

### Question 1. Increase to 1024 Neurons -- What's the impact?

1. Training takes longer, but is more accurate
2. Training takes longer, but no impact on accuracy
3. Training takes the same time, but is more accurate


#### Answer
The correct answer is (1) by adding more Neurons we have to do more calculations, slowing down the process, but in this case they have a good impact -- we do get more accurate. That doesn't mean it's always a case of 'more is better', you can hit the law of diminishing returns very quickly!

## Exercise 3: 

What would happen if you remove the Flatten() layer. Why do you think that's the case? 

You get an error about the shape of the data. It may seem vague right now, but it reinforces the rule of thumb that the first layer in your network should be the same shape as your data. Right now our data is 28x28 images, and 28 layers of 28 neurons would be infeasible, so it makes more sense to 'flatten' that 28,28 into a 784x1. Instead of wriitng all the code to handle that ourselves, we add the Flatten() layer at the begining, and when the arrays are loaded into the model later, they'll automatically be flattened for us.

In [103]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([#tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

training_images = training_images.reshape(len(training_images), -1)
test_images = test_images.reshape(len(test_images), -1)

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 1s 703us/step - loss: 0.2998 - accuracy: 0.9152
Epoch 2/5
1875/1875 [==============================] - 1s 697us/step - loss: 0.1487 - accuracy: 0.9562
Epoch 3/5
1875/1875 [==============================] - 1s 710us/step - loss: 0.1076 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 1s 680us/step - loss: 0.0857 - accuracy: 0.9742
Epoch 5/5
313/313 [==============================] - 0s 537us/step - loss: 0.0897 - accuracy: 0.9703


[0.08972948044538498, 0.970300018787384]

## Exercise 4: 

Consider the final (output) layers. Why are there 10 of them? What would happen if you had a different amount than 10? For example, try training the network with 5

You get an error as soon as it finds an unexpected value. Another rule of thumb -- the number of neurons in the last layer should match the number of classes you are classifying for. In this case it's the digits 0-9, so there are 10 of them, hence you should have 10 neurons in your final layer.

In [108]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 1s 718us/step - loss: 0.3115 - accuracy: 0.9126
Epoch 2/5
1875/1875 [==============================] - 1s 692us/step - loss: 0.1524 - accuracy: 0.9559
Epoch 3/5
1875/1875 [==============================] - 1s 694us/step - loss: 0.1105 - accuracy: 0.9676
Epoch 4/5
1875/1875 [==============================] - 1s 722us/step - loss: 0.0876 - accuracy: 0.9738
Epoch 5/5
313/313 [==============================] - 0s 595us/step - loss: 0.0903 - accuracy: 0.9722


[0.0903458371758461, 0.9721999764442444]

## Exercise 5: 

Consider the effects of additional layers in the network. What will happen if you add another layer between the one with 512 and the final layer with 10. 

Ans: There isn't a significant impact -- because this is relatively simple data. For far more complex data (including color images to be classified as flowers that you'll see in the next lesson), extra layers are often necessary. 

In [42]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1855 - accuracy: 0.9439
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0818 - accuracy: 0.9738
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0544 - accuracy: 0.9834
Epoch 4/5
1875/1875 [==============================] - ETA: 0s - loss: 0.0431 - accuracy: 0.98 - 5s 3ms/step - loss: 0.0432 - accuracy: 0.9863
Epoch 5/5
313/313 [==============================] - 0s 824us/step - loss: 0.0770 - accuracy: 0.9804


[0.07703782618045807, 0.980400025844574]

# Exercise 6: 

Consider the impact of training for more or less epochs. Why do you think that would be the case? 

Try 15 epochs -- you'll probably get a model with a much better loss than the one with 5
Try 30 epochs -- you might see the loss value stops decreasing, and sometimes increases. This is a side effect of something called 'overfitting' which you can learn about [somewhere] and it's something you need to keep an eye out for when training neural networks. There's no point in wasting your time training if you aren't improving your loss, right! :)

In [43]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=30)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/30
1875/1875 [==============================] - 1s 704us/step - loss: 0.2613 - accuracy: 0.9253
Epoch 2/30
1875/1875 [==============================] - 1s 705us/step - loss: 0.1159 - accuracy: 0.9662
Epoch 3/30
1875/1875 [==============================] - 1s 698us/step - loss: 0.0798 - accuracy: 0.9758
Epoch 4/30
1875/1875 [==============================] - 1s 709us/step - loss: 0.0596 - accuracy: 0.9816
Epoch 5/30
1875/1875 [==============================] - 1s 708us/step - loss: 0.0470 - accuracy: 0.9851
Epoch 6/30
1875/1875 [==============================] - 1s 703us/step - loss: 0.0355 - accuracy: 0.9890
Epoch 7/30
1875/1875 [==============================] - 1s 702us/step - loss: 0.0299 - accuracy: 0.9902
Epoch 8/30
1875/1875 [==============================] - 1s 705us/step - loss: 0.0232 - accuracy: 0.9929
Epoch 9/30
1875/1875 [==============================] - 1s 717us/step - loss: 0.0187 - accuracy: 0.9941
Epoch 10/30
1875/1875 [==============================] - 1

[0.1275814026594162, 0.9778000116348267]

# Exercise 7: 

Before you trained, you normalized the data, going from values that were 0-255 to values that were 0-1. What would be the impact of removing that? Here's the complete code to give it a try. Why do you think you get different results? 

In [109]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.7995 - accuracy: 0.9079
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3499 - accuracy: 0.9360
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3007 - accuracy: 0.9396
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2602 - accuracy: 0.9464
Epoch 5/5
313/313 [==============================] - 0s 750us/step - loss: 0.2633 - accuracy: 0.9484


[0.2633044719696045, 0.9484000205993652]

# Exercise 8: 

Earlier when you trained for extra epochs you had an issue where your loss might change. It might have taken a bit of time for you to wait for the training to do that, and you might have thought 'wouldn't it be nice if I could stop the training when I reach a desired value?' -- i.e. 95% accuracy might be enough for you, and if you reach that after 3 epochs, why sit around waiting for it to finish a lot more epochs....So how would you fix that? Like any other program...you have callbacks! Let's see them in action...

Refer to https://www.tensorflow.org/guide/keras/custom_callback for custom callbacks

In [111]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.4):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

model.evaluate(test_images, test_labels)

2.3.0
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4741 - accuracy: 0.8317
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3603 - accuracy: 0.8687
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3245 - accuracy: 0.8800
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3018 - accuracy: 0.8876
Epoch 5/5
313/313 [==============================] - 0s 771us/step - loss: 0.3480 - accuracy: 0.8722


[0.3479938209056854, 0.8722000122070312]